In [3]:
import pygetwindow as gw
import psutil
import win32com.client
import os
import time
import subprocess
import PySimpleGUI as sg
import win32clipboard
import pandas as pd
from datetime import date, datetime, timedelta
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl
import sys

# Funções

# ACESSO SAP
def acessar_sap():
    
    try:
        # Encontra o processo do SAP e o termina
        for appsap in psutil.process_iter():
            if appsap.name() == "saplogon.exe":
                appsap.terminate()
                print('SAP reiniciado.')

        print('Acessando o SAP...')
        
        path = r"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe"
        subprocess.Popen(path)
        time.sleep(5)
        SapGuiAuto = win32com.client.GetObject('SAPGUI')

        if not type(SapGuiAuto) == win32com.client.Dispatch:

            application = SapGuiAuto.GetScriptingEngine
            connection = application.OpenConnection("S/4HANA - PRD (SSO)", True)
            time.sleep(3)
            session = connection.Children(0)
        # Encontre a janela do SAP Logon com base no título da janela
        janela = gw.getWindowsWithTitle('SAP Logon')[0]
        # Minimize a janela
        janela.minimize()
        # Minizar janela
        session.findById("wnd[0]").iconify()
        
        print('SAP Conectado!\n')
        
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

# abrir nova janela no sap
def abrir_nova_sessao():
    
    SapGuiAuto = win32com.client.GetObject('SAPGUI')
    application = SapGuiAuto.GetScriptingEngine
    connection = application.Children(0)
    session = connection.Children(0)

    #abrir nova janela
    session.createSession()
    
    session.findById("wnd[0]").iconify()
    
    print('SAP Conectado!')
                
# verificar janelas do sap
def verificar_sap():
    
    windows = gw.getAllTitles()
    janela_prod = False

    for window in windows:
        if 'PRD' in window:
            janela_prod = True
    
    return janela_prod
    
# Extração da MB51 # df_mb51
def ext_mb51():
    SapGuiAuto = win32com.client.GetObject('SAPGUI')
    application = SapGuiAuto.GetScriptingEngine
    connection = application.Children(0)
    session = connection.Children(0)
    
    print('\nExtração da MB51 iniciada...')

    # Acessar caixa de pesquisa da função e digitar
    session.findById("wnd[0]/tbar[0]/okcd").text = "/NMB51"
    # Clicar em enter
    session.findById("wnd[0]").sendVKey(0)

    # Abre o campo Material e limpa
    session.findById("wnd[0]/usr/btn%_MATNR_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    
    # Abre o campo Centro e Limpa
    session.findById("wnd[0]/usr/btn%_WERKS_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()

    # Loop para adicionar loja e linha
    for loja in lojas:
        # Comando para adicionar loja
        session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").text = loja
        # Comando para adicionar linha
        session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").caretPosition = 4
        session.findById("wnd[1]/tbar[0]/btn[13]").press()
    
    # Comando para dar F8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()

    # Abre o campo Deposito/Limpar campo/Cola dados copiado/Clica em OK
    session.findById("wnd[0]/usr/ctxtLGORT-LOW").text = "DP10"
    
    # Abre o tipo de movimento e limpa todos
    session.findById("wnd[0]/usr/btn%_BWART_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()

    # Insere a data inicial
    session.findById("wnd[0]/usr/ctxtBUDAT-LOW").Text = '19082024'

    # Insere a data final
    session.findById("wnd[0]/usr/ctxtBUDAT-HIGH").Text = '10092024'

    # Insere o layout desejado
    session.findById("wnd[0]/usr/ctxtALV_DEF").Text = "/91343"

    # Executa a transação
    session.findById("wnd[0]/tbar[1]/btn[8]").press()

    # exporta a transação como ClipBoard
    session.findById("wnd[0]/mbar/menu[0]/menu[1]/menu[2]").select()
    session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[4,0]").select()
    session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[4,0]").setFocus()
    session.findById("wnd[1]/tbar[0]/btn[0]").press()

    # Get to ClipBoard
    win32clipboard.OpenClipboard()
    df_mb51 = win32clipboard.GetClipboardData()
    win32clipboard.CloseClipboard()

    # Lendo recuperando cliboard em dataframe, remmovendo linhas em branco de cabeçalho, pondo em UTF-8
    df_mb51 = pd.read_clipboard(
        sep='|',
        skiprows = range(0,3), 
        on_bad_lines = 'skip', 
        encoding = 'UTF-8',
        converters={'Cen.': str}
    )

    # Removendo espaços
    df_mb51.columns = df_mb51.columns.str.strip()

    # Selecionando colunas necessarias
    df_mb51 = df_mb51[['Material',
            'Cen.',
            'Dep.',
            'TMv',
            'Doc.mat.',
            'Data lçto.',
            'Qtd.',
            'Usuário',
            'D/C',
            'Data doc.',
            'Pedido',
            'Referência']]

    # Renomeando as colunas de forma padrão xlsx
    df_mb51.rename(columns={'Cen.': 'Centro'}, inplace=True)
    df_mb51.rename(columns={'Dep.': 'Depósito'}, inplace=True)
    df_mb51.rename(columns={'TMv': 'Tipo de movimento'}, inplace=True)
    df_mb51.rename(columns={'Doc.mat.': 'Doc.material'}, inplace=True)
    df_mb51.rename(columns={'Data lçto.': 'Data de lançamento'}, inplace=True)
    df_mb51.rename(columns={'Qtd.': 'Qtd.  UM registro'}, inplace=True)
    df_mb51.rename(columns={'Usuário': 'Nome do usuário'}, inplace=True)
    df_mb51.rename(columns={'D/C': 'Cód.débito/crédito'}, inplace=True)
    df_mb51.rename(columns={'Data doc.': 'Data do documento'}, inplace=True)

    # formatando
    df_mb51 = df_mb51.drop(0, axis=0)
    df_mb51 = df_mb51.dropna()
    df_mb51['Material'] = df_mb51['Material'].astype('object')
    df_mb51['Depósito'] = df_mb51['Depósito'].astype('object')
    df_mb51['Doc.material'] = df_mb51['Doc.material'].astype('int64')
    df_mb51['Data de lançamento'] = pd.to_datetime(df_mb51['Data de lançamento'], dayfirst=True)
    
    # Tratamento da coluna 'Qtd.  UM registro'
    df_mb51['Qtd.  UM registro'] = (df_mb51['Qtd.  UM registro'].astype(str).str.strip()
    .apply(lambda x: '-' + x[:-1].replace('.', '').replace(',', '.') if x.endswith('-') else x.replace('.', '').replace(',', '.'))
    .astype(float)
    )

    
    # Converter outras colunas
    df_mb51['Nome do usuário'] = df_mb51['Nome do usuário'].astype('object')
    df_mb51['Cód.débito/crédito'] = df_mb51['Cód.débito/crédito'].astype('object')
    df_mb51['Data do documento'] = pd.to_datetime(df_mb51['Data do documento'], dayfirst=True)
    df_mb51['Pedido'] = df_mb51['Pedido'].astype('object')
    df_mb51['Referência'] = df_mb51['Referência'].astype('object')

    # Salvar em arquivo Excel
    df_mb51.to_excel(r'Bases\MB51.xlsx', index=False)

    print('Extração da MB51 finalizada com sucesso! ;D')
    return df_mb51

# Extração da NF pendentes # df_nfp
def ext_nfp():
    
    try:
    
        SapGuiAuto = win32com.client.GetObject('SAPGUI')
        application = SapGuiAuto.GetScriptingEngine
        connection = application.Children(0)
        session = connection.Children(0)
                
        print('\nExtração NFs pendentes iniciada...')

        # acessando a transação da NFs pendentes
        session.findById("wnd[0]").iconify()
        session.findById("wnd[0]/tbar[0]/okcd").text = "/NZTMM046"
        session.findById("wnd[0]").sendVKey(0)

        # filtrando lojas e exportando nf pendentes
        session.findById("wnd[0]/usr/btn%_S_WERKS_%_APP_%-VALU_PUSH").press()
        session.findById("wnd[1]/tbar[0]/btn[16]").press()

        # Loop para adicionar loja e linha
        for loja in lojas:
            # Comando para adicionar loja
            session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").text = loja
            # Comando para adicionar linha
            session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").caretPosition = 4
            session.findById("wnd[1]/tbar[0]/btn[13]").press()

        # F8
        session.findById("wnd[1]/tbar[0]/btn[8]").press()
        session.findById("wnd[0]/tbar[1]/btn[8]").press()
        
        
        # clique e salvando
        session.findById("wnd[0]/usr/tabsTS_9000/tabpTS_9000_FC1/ssubTS_9000_SCA:ZRMM_CONSULTA_NFE_ENTRADA:9100/cntlCC_HEADER/shellcont/shell").setCurrentCell(3, "EBELN")
        session.findById("wnd[0]/usr/tabsTS_9000/tabpTS_9000_FC1/ssubTS_9000_SCA:ZRMM_CONSULTA_NFE_ENTRADA:9100/cntlCC_HEADER/shellcont/shell").selectedRows = "3"
        session.findById("wnd[0]/usr/tabsTS_9000/tabpTS_9000_FC1/ssubTS_9000_SCA:ZRMM_CONSULTA_NFE_ENTRADA:9100/cntlCC_HEADER/shellcont/shell").clickCurrentCell()
        session.findById("wnd[0]/usr/tabsTS_9000/tabpTS_9000_FC1/ssubTS_9000_SCA:ZRMM_CONSULTA_NFE_ENTRADA:9100/cntlCC_ITEM_DET/shellcont/shell").pressToolbarButton("EXCEL")

        # buscando o arquivo mais recente na pasta TEMP
        caminho_temp = 'C:\TEMP'
        lista_temp = os.listdir(caminho_temp)

        lista_datas_temp = []
        for arquivo in lista_temp:
            # descobrir data desse arquivo
            data_temp = os.path.getmtime(f"{caminho_temp}/{arquivo}")
            lista_datas_temp.append((data_temp, arquivo))

        lista_datas_temp.sort(reverse=True)
        nome_do_arquivo = lista_datas_temp[0][1]

        # lendo arquivo da nf pendente baixado
        df_nfp = pd.read_csv(f'{caminho_temp}\\{nome_do_arquivo}', encoding='latin1', sep=';')

        # Salvando o arquivo como csv
        df_nfp.to_csv(rf'Bases\NFP.csv', index=False, encoding='latin1')

        print("Extração das NFs pendentes realizado com sucesso!  ;)")
    
    except:
    
        # Definindo as colunas
        colunas = ['ID SAP', 'Origem', 'Destino', 'NF', 'Serie', 'Valor NF', 'Material',
            'Descrição', 'Grupo', 'Categoria', 'UM', 'Quantidade Pedido',
            'Quantidade Recebida', 'Data NF', 'Data Lancto', 'Data de Recebimento',
            'Usuario Recebimento', 'Situação', 'Custo Empresa', 'Custo Fiscal',
            'Previsão Entrega', 'Modelo', 'Cod.UDN', 'Pedido', 'Item', 'ICMS',
            'Custo Uni NF']

        # Criando o DataFrame vazio com as colunas definidas
        df_nfp = pd.DataFrame(columns=colunas)
        
        print("Lojas sem NFs Pendentes...")
    
    return df_nfp

# Extração da DIVERGENCIA DE RECEBIMENTO # df_drecebimento
def ext_dreceb():
    # Divergência de Recebimento
    SapGuiAuto = win32com.client.GetObject('SAPGUI')
    application = SapGuiAuto.GetScriptingEngine
    connection = application.Children(0)
    session = connection.Children(0)

    print('\nExtração Divergências de Recebimento iniciada...')

    # acessando a transação da NFs pendentes
    session.findById("wnd[0]").iconify()
    session.findById("wnd[0]/tbar[0]/okcd").text = "/NZTMM128"
    session.findById("wnd[0]").sendVKey(0)

    # Filtrando as informações para extração
    session.findById("wnd[0]/usr/btn%_S_WERKS_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[16]").press()

    # Loop para adicionar loja e linha
    for loja in lojas:
        # Comando para adicionar loja
        session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").text = loja
        # Comando para adicionar linha
        session.findById("wnd[1]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/ctxtRSCSEL_255-SLOW_I[1,0]").caretPosition = 4
        session.findById("wnd[1]/tbar[0]/btn[13]").press()

    # F8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()

    # DATAS
    session.findById("wnd[0]/usr/ctxtS_BUDAT-LOW").text = data_inicial
    session.findById("wnd[0]/usr/ctxtS_BUDAT-HIGH").text = data_final
    session.findById("wnd[0]/usr/ctxtS_BUDAT-HIGH").setFocus()
    session.findById("wnd[0]/usr/ctxtS_BUDAT-HIGH").caretPosition = 8
    session.findById("wnd[0]/tbar[1]/btn[8]").press()

    # SELECIONANDO LAYOUT
    session.findById("wnd[0]/usr/shell/shellcont/shell").pressToolbarContextButton("&MB_VARIANT")
    session.findById("wnd[0]/usr/shell/shellcont/shell").selectContextMenuItem("&LOAD")
    session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").selectedRows = "0"
    session.findById("wnd[1]/usr/ssubD0500_SUBSCREEN:SAPLSLVC_DIALOG:0501/cntlG51_CONTAINER/shellcont/shell").clickCurrentCell()

    # EXPORTANDO
    session.findById("wnd[0]/usr/shell/shellcont/shell").pressToolbarContextButton("&MB_EXPORT")
    session.findById("wnd[0]/usr/shell/shellcont/shell").selectContextMenuItem("&PC")
    session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[4,0]").select()
    session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[4,0]").setFocus()
    session.findById("wnd[1]/tbar[0]/btn[0]").press()

    # Get to ClipBoard
    win32clipboard.OpenClipboard()
    df_drecebimento = win32clipboard.GetClipboardData()
    win32clipboard.CloseClipboard()

    # Recebendo o arquivo copiado com clipboard
    df_drecebimento = pd.read_clipboard(
            sep='|',
            skiprows = range(0,3), 
            on_bad_lines = 'skip', 
            encoding = 'UTF-8',
            converters={'Cen.': str},
        )

    # Lista com os índices das colunas que deseja selecionar
    indices_colunas = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

    # Selecionando as colunas pelo índice
    df_drecebimento = df_drecebimento.iloc[:, indices_colunas]

    # Renomeando as colunas pelo índice
    df_drecebimento = df_drecebimento.rename(columns={df_drecebimento.columns[0]: 'Doc.material'})
    df_drecebimento = df_drecebimento.rename(columns={df_drecebimento.columns[1]: 'Data de lançamento'})
    df_drecebimento = df_drecebimento.rename(columns={df_drecebimento.columns[2]: 'Centro'})
    df_drecebimento = df_drecebimento.rename(columns={df_drecebimento.columns[3]: 'Depósito'})
    df_drecebimento = df_drecebimento.rename(columns={df_drecebimento.columns[6]: 'Texto breve material'})
    df_drecebimento = df_drecebimento.rename(columns={df_drecebimento.columns[8]: 'Unid.medida básica'})
    df_drecebimento = df_drecebimento.rename(columns={df_drecebimento.columns[9]: 'Motivo do movimento'})
    df_drecebimento = df_drecebimento.rename(columns={df_drecebimento.columns[10]: 'Motivo do movimento'})
    df_drecebimento = df_drecebimento.rename(columns={df_drecebimento.columns[11]: 'Tratado'})

    # Removendo linhas vazias
    df_drecebimento = df_drecebimento.dropna()

    # Convertendo texto para data
    df_drecebimento['Data de lançamento'] = pd.to_datetime(df_drecebimento['Data de lançamento'], dayfirst=True)

    # Salvando arquivo como xlsx
    df_drecebimento.to_excel(r"Bases\DRECEB.xlsx", index=False)

    print('Extração Divergências de Recebimento finalizada com sucesso :D')
    
    return df_drecebimento

# Extração da ZTMM003 # df_inventario
def ext_ztmm003(loja_analisada):
    
    try:
        # conectando com o SAP
        SapGuiAuto = win32com.client.GetObject('SAPGUI')
        application = SapGuiAuto.GetScriptingEngine
        connection = application.Children(0)
        session = connection.Children(0)

        print('\nExtração ZTMM003 em andamento...')

        # acessando a transação da NFs pendentes
        session.findById("wnd[0]").iconify()
        session.findById("wnd[0]/tbar[0]/okcd").text = "/NZTMM003"
        session.findById("wnd[0]").sendVKey(0)

        # Filtrando as informações para extração

        session.findById("wnd[0]/usr/ctxtS_BLDAT-LOW").text = data_hoje
        session.findById("wnd[0]/usr/ctxtS_BLDAT-HIGH").text = data_hoje
        session.findById("wnd[0]/usr/ctxtS_CENTRO-LOW").text = loja_analisada
        session.findById("wnd[0]/usr/ctxtS_CENTRO-HIGH").text = loja_analisada 
        
        
        session.findById("wnd[0]/usr/ctxtS_CENTRO-HIGH").setFocus()
        session.findById("wnd[0]/usr/ctxtS_CENTRO-HIGH").caretPosition = 4
        session.findById("wnd[0]/tbar[1]/btn[8]").press()
        session.findById("wnd[0]/tbar[1]/btn[33]").press()
        session.findById("wnd[1]/usr/cntlGRID/shellcont/shell").currentCellRow = 13
        session.findById("wnd[1]/usr/cntlGRID/shellcont/shell").selectedRows = "13"
        session.findById("wnd[1]/usr/cntlGRID/shellcont/shell").clickCurrentCell()
        session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell").setCurrentCell(11,"MAKTG")
        session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell").selectedRows = "11"
        session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell").contextMenu()
        session.findById("wnd[0]/mbar/menu[0]/menu[3]/menu[2]").select()
        session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[4,0]").select()
        session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[4,0]").setFocus()
        session.findById("wnd[1]/tbar[0]/btn[0]").press()
            
        # Get to ClipBoard
        win32clipboard.OpenClipboard()
        df_inventario = win32clipboard.GetClipboardData()
        win32clipboard.CloseClipboard()

        df_inventario = pd.read_clipboard(
                sep='|',
                skiprows = range(0,3), 
                on_bad_lines = 'skip', 
                encoding = 'UTF-8',
                converters={'Material': str,
                            'EAN': str,
                            'Descr. Material': str,
                            'Categoria': str,
                            'Grupo': str                                                                                          
                            }
            )
        
        # Remove a primeira e última linha
        df_inventario = df_inventario.iloc[1:-1]

        # Remove a primeira e última coluna
        df_inventario = df_inventario.iloc[:, 1:-1]
        
        # Remove os espaços do cabeçalhos
        df_inventario.columns = df_inventario.columns.str.strip()
        
        # Extrai o texto da direita até o (-) hifen
        df_inventario['Grupo'] = df_inventario['Grupo'].str.split('-').str[1].str.strip()
        
        
        # Pega todos os cabeçalhos e coloca em lista 
        cabecalhos = list(df_inventario.columns.values)

        for i in range(5, 6):       
            # Limpar os espaços em cada linha
            df_inventario[cabecalhos[i]] = df_inventario[cabecalhos[i]].str.strip()
        
        for i in range(8, 10):       
            # Limpar os espaços em cada linha
            df_inventario[cabecalhos[i]] = df_inventario[cabecalhos[i]].str.strip()

        for i in range(22, 28):       
            # Limpar os espaços em cada linha
            df_inventario[cabecalhos[i]] = df_inventario[cabecalhos[i]].str.strip()

            # inverte o sinal negativo da direita para a esquerda
            df_inventario[cabecalhos[i]] = df_inventario[cabecalhos[i]].astype(str).apply(lambda x: '-' + x[:-1].strip() if x[-1] == '-' else x)
            
        for i in range(22, 24):
            # Converter as colunas em numeros inteiros
            df_inventario[cabecalhos[i]] = df_inventario[cabecalhos[i]].astype('int64')

        # for i in range(24, 28):
        #     # Converter as colunas em numeros fracionados
        #     df_inventario[cabecalhos[i]] = df_inventario[cabecalhos[i]].str.replace(',', '.').astype('float64')
        
        for i in range(24, 28):
            # Remove os pontos de milhar e substitua a vírgula por ponto para salvar como numero
            df_inventario[cabecalhos[i]] = pd.to_numeric(df_inventario[cabecalhos[i]].str.replace('.', '').str.replace(',', '.'))
        
        # Salvar o arquivo como backup
        df_inventario.to_excel(f"Bases/ZTMM003_{loja_analisada}_{data_hoje}.xlsx", index=False)
        
        negativos = ((df_inventario['Qtd. Contabilistica'] < 0))
                                        
        itens_negativos = df_inventario[negativos]

        itens_negativos = itens_negativos[['Material', 'Descr. Material', 'Qtd. Contabilistica']]
        
        print('\nExtração ZTMM003 finalizada...')
    
    except:
        print("Não existem dados para a seleção")
    
    return df_inventario, itens_negativos

# GERAR MACROS    
def gerar_macros():    
    # Itera sobre cada loja do analista
    print("\nGeração das macros iniciada...\n")
    for loja in lojas:
    
        # Filtra os dataframes com a loja do analista
        df_mb51_loja = df_mb51[df_mb51['Centro'] == loja]
        df_nfp_loja = df_nfp[df_nfp['Destino'] == int(loja)]
        df_diverg_loja = df_rec[df_rec['Centro'] == loja]
        
        # Carrega a planilha modelo
        wb = openpyxl.load_workbook(r'Bases\Pre_Macro_Inventarios_v9.xlsm', keep_vba=True)
        
        # Escreve os dados filtrados na planilha nas guias correspondentes
        ws_mb51 = wb["MB51"]
        ws_nfp = wb["NF Pendente"]
        ws_diverg = wb["Divergência de Recebimento"]

        # Função para copiar os dados de um DataFrame para uma aba do Excel
        def dataframe_to_sheet(ws, df):
            rows = dataframe_to_rows(df, index=False, header=True)
            for r_idx, row in enumerate(rows, 1):
                for c_idx, value in enumerate(row, 1):
                    ws.cell(row=r_idx, column=c_idx, value=value)

        # Copia e cola os dados filtrados nas abas correspondentes
        dataframe_to_sheet(ws_mb51, df_mb51_loja)
        dataframe_to_sheet(ws_nfp, df_nfp_loja)
        dataframe_to_sheet(ws_diverg, df_diverg_loja)
        
        hora = datetime.now().strftime('%H_%M')

        # Salva o arquivo .xlsm com os dados da loja do analista
        
        wb.save(rf'Macros\{loja}_{data_hoje}_{hora}.xlsm')
        wb.close()
        print(f'Macro da loja {loja} foi criada.')   
    print('\nMacro(s) criada(s) com sucesso! ;D')

# PEGAR VALORES LOJAS
def get_input(values):  
    user_input = values['input']
    if len(user_input) == 4 and user_input.isdigit():
        return user_input
    else:
        return None

# Tratando todas as bases
def tratar_bases(df_inventario, df_nfp, df_mojix, df_mb51, df_drecebimento):
    
    print('Analise Iniciada :D')
    
    # adicionando coluna 'movimento' com os dados 'Diferença Inventario'
    df_inventario['Movimento'] = 'Diferenca Inventario'

    # tratamento da coluna Material, tirando os caracteres desnecessarios
    df_nfp['Material'] = df_nfp['Material'].str.replace('"', '').str.replace('=', '')

    # convertendo coluna texto para data
    df_mojix['Encoded Date'] = pd.to_datetime(df_mojix['Encoded Date'], format='%d/%m/%Y', errors='coerce') 
    
    

    #----------------------------------------------------------------------------------------------------------------------------------

    # Aplicando os filtros necessários

    # Filtrando a loja informada pelo usuario
    filtro_loja_nfp = df_nfp.loc[df_nfp['Destino'].astype(int) == int(loja_analisada)]
    filtro_loja_mb51 = df_mb51.loc[df_mb51['Centro'].astype(int) == int(loja_analisada)]
    filtro_loja_rec = df_drecebimento.loc[df_drecebimento['Centro'].astype(int) == int(loja_analisada)]
    filtro_loja_mojix = df_mojix[df_mojix['Store ID'] == int(loja_analisada)]

    # Filtrando a data informada pelo usuario
    filtro_loja_mb51 = filtro_loja_mb51.loc[filtro_loja_mb51['Data de lançamento'] >= pd.to_datetime(data_analisada, format='%d/%m/%Y')]
    filtro_loja_rec = filtro_loja_rec.loc[filtro_loja_rec["Data de lançamento"] >= pd.to_datetime(data_analisada, format='%d/%m/%Y')]
    filtro_loja_mojix = filtro_loja_mojix.loc[filtro_loja_mojix["Encoded Date"] >= data_analisada]

    #----------------------------------------------------------------------------------------------------------------------------------

    # Trantando base NF Pendentes

    # Lista com os índices das colunas que irá selecionar
    indices_colunas_nfp = [2, 6, 11]

    # Selecionando as colunas pelo índice
    filtro_loja_nfp = filtro_loja_nfp.iloc[:, indices_colunas_nfp]

    # Renomeando colunas
    filtro_loja_nfp = filtro_loja_nfp.rename(columns={filtro_loja_nfp.columns[0]: 'Centro'})
    filtro_loja_nfp = filtro_loja_nfp.rename(columns={filtro_loja_nfp.columns[1]: 'Material'})
    filtro_loja_nfp = filtro_loja_nfp.rename(columns={filtro_loja_nfp.columns[2]: 'Qtde'})

    filtro_loja_nfp['Movimento'] = 'NF Pendente'

    #----------------------------------------------------------------------------------------------------------------------------------
    # Trantando base MB51

    # Lista com os índices das colunas que irá selecionar
    indices_colunas_mb51 = [0, 1, 3, 6, 11]

    # Selecionando as colunas pelo índice
    filtro_loja_mb51 = filtro_loja_mb51.iloc[:, indices_colunas_mb51]

    # Renomeando colunas
    filtro_loja_mb51 = filtro_loja_mb51.rename(columns={filtro_loja_mb51.columns[0]: 'Material'})
    filtro_loja_mb51 = filtro_loja_mb51.rename(columns={filtro_loja_mb51.columns[2]: 'Movimento'})
    filtro_loja_mb51 = filtro_loja_mb51.rename(columns={filtro_loja_mb51.columns[3]: 'Qtde'})

    # Converte a coluna 'Movimento' para string e remova '.0' se existir
    filtro_loja_mb51['Movimento'] = filtro_loja_mb51['Movimento'].astype(str).str.replace('.0', '', regex=False)

    # Renomeando os tipos de movimentos
    filtro_loja_mb51['Movimento'] = filtro_loja_mb51['Movimento'].replace(['701', '702'], 'Inventarios')
    filtro_loja_mb51['Movimento'] = filtro_loja_mb51['Movimento'].replace(['601', '602'], 'OMS')
    filtro_loja_mb51['Movimento'] = filtro_loja_mb51['Movimento'].replace(['251', '252'], 'Vendas')
    filtro_loja_mb51['Movimento'] = filtro_loja_mb51['Movimento'].replace(['653', '654'], 'Devolução Cliente')
    filtro_loja_mb51['Movimento'] = filtro_loja_mb51['Movimento'].replace(['101'], 'Recebimento Fornecedor')
    filtro_loja_mb51['Movimento'] = filtro_loja_mb51['Movimento'].replace(['861'], 'Recebimento CD/LOJA')
    filtro_loja_mb51['Movimento'] = filtro_loja_mb51['Movimento'].replace(['862'], 'Saída de Transferência')
    filtro_loja_mb51['Movimento'] = filtro_loja_mb51['Movimento'].replace(['ZET'], 'ZET')


    #---------------------------------------------------------------------------------------------------------------------------------
    # Trantando base Divergencia de Recebimento

    # Renomeando os tipos de movimentos
    filtro_loja_rec['Motivo do movimento'] = filtro_loja_rec['Motivo do movimento'].replace([int('30'), int('40')], 'Perda Diverg. Receb.')
    filtro_loja_rec['Motivo do movimento'] = filtro_loja_rec['Motivo do movimento'].replace([int('110'), int('120')], 'Ganho Diverg. Receb.')

    # Lista com os índices das colunas que irá selecionar
    indices_colunas_rec = [2, 5, 7, 9]

    # Selecionando as colunas pelo índice
    filtro_loja_rec = filtro_loja_rec.iloc[:, indices_colunas_rec]

    # Renomeando colunas
    filtro_loja_rec = filtro_loja_rec.rename(columns={filtro_loja_rec.columns[3]: 'Movimento'})
    filtro_loja_rec = filtro_loja_rec.rename(columns={filtro_loja_rec.columns[2]: 'Qtde'})
    filtro_loja_rec = filtro_loja_rec.rename(columns={filtro_loja_rec.columns[1]: 'Material'})

    #---------------------------------------------------------------------------------------------------------------------------------------
    # Trantando base Mojix

    # Lista com os índices das colunas que irá selecionar
    indices_colunas_mojix = [3, 6]

    # Selecionando as colunas pelo índice
    filtro_loja_mojix = filtro_loja_mojix.iloc[:, indices_colunas_mojix]

    # Renomeando colunas
    filtro_loja_mojix = filtro_loja_mojix.rename(columns={filtro_loja_mojix.columns[0]: 'Material'})
    filtro_loja_mojix = filtro_loja_mojix.rename(columns={filtro_loja_mojix.columns[1]: 'Centro'})

    filtro_loja_mojix['Movimento'] = 'Mojix'
    filtro_loja_mojix['Qtde'] = 1
    
    # Adiciona coluna DUPLICADOS se é verdadeiro ou falso
    df_inventario['DUPLICADOS'] = (df_inventario['Qtd. Diferença'] - df_inventario['Qtd. Contabilistica'] == 0) & \
                                ~((df_inventario['Qtd. Diferença'] == 0) & (df_inventario['Qtd. Contabilistica'] == 0))
                                
 

    return filtro_loja_nfp, filtro_loja_mb51, filtro_loja_rec, filtro_loja_mojix, df_inventario 

def filtros_manter_estoque_1(df_inventario):
    
    condicao13 = (((df_inventario['Grupo'] == "SERVICOS") | (df_inventario['Grupo'] == "EMBALAGENS")) &
            ((df_inventario['Categoria'] == "MARKETING") | (df_inventario['Categoria'] == "TODAS")) &
            ((df_inventario['Descr. Material'].str.contains("CARTAO PRESENTE")) | 
            (df_inventario['Descr. Material'].str.contains("BERÇO"))))
    
    negativos = ((df_inventario['Qtd. Contabilistica'] < 0))
    
    return condicao13, negativos

def receber_manter_estoque_1(condicao13, arquivo_contag_manual):
    
    arquivo_txt = df_inventario[condicao13]
    
    arquivo_txt = pd.concat([arquivo_txt, arquivo_contag_manual])
    

    
    return arquivo_txt

# filtrando o que for para manter estoque
def filtros_manter_estoque_0(df_inventario):
    
    # Filtros para caso NÃO tiver contagem manual
    condicao1 = ((df_inventario['Grupo'] == 'ACESSORIOS') &
                (df_inventario['Categoria'] == 'FUTEBOL') &
                (df_inventario['Descr. Material'].str.contains('CAMPO AEROTRACK', case=False) |
                df_inventario['Descr. Material'].str.contains('SOFT X', case=False) |
                df_inventario['Descr. Material'].str.contains('SHOW DE BOLA', case=False) |
                df_inventario['Descr. Material'].str.contains('ALBUM', case=False) |
                df_inventario['Descr. Material'].str.contains('MINI BOLA', case=False)))

    condicao2 = ((df_inventario['Grupo'] == 'ACESSORIOS') &
                (df_inventario['Categoria'] == 'TREINO') &
                (df_inventario['Descr. Material'].str.contains('BOLA TURMA', case=False) |
                df_inventario['Descr. Material'].str.contains('BOLA VOLEI', case=False) |
                df_inventario['Descr. Material'].str.contains('BOLA DE BASQUETE', case=False) |
                df_inventario['Descr. Material'].str.contains('BOLA EURO', case=False)))

    condicao3 = ((df_inventario['Grupo'] == 'ACESSORIOS') &
                (df_inventario['Categoria'] == 'TODAS') &
                (df_inventario['Descr. Material'].str.contains('CAIXA DE PRESENTE OXER', case=False)))

    condicao4 = ((df_inventario['Grupo'] == 'ACESSORIOS') &
                (df_inventario['Categoria'].isin(['TODAS', 'CHECKSTAND'])) &
                (df_inventario['Descr. Material'].str.contains('TENYS PE', case=False) |
                df_inventario['Descr. Material'].str.contains('CUID PINK', case=False)))

    condicao5 = ((df_inventario['Grupo'] == 'ACESSORIOS') &
                (df_inventario['Categoria'].isin(['TREINO', 'CHECKSTAND', 'CHECKSTAND - ACESSORIOS', 'TODAS'])) &
                (df_inventario['Descr. Material'].str.contains('EMBALAG', case=False) |
                df_inventario['Descr. Material'].str.contains('KINESIO', case=False)))

    condicao6 = ((df_inventario['Grupo'] == 'EQUIPAMENTOS') &
                (df_inventario['Categoria'].isin(['PERFORMANCE', 'TREINO'])) &
                (df_inventario['Descr. Material'].str.contains('ANILHA', case=False) |
                df_inventario['Descr. Material'].str.contains('BARRA', case=False) |
                df_inventario['Descr. Material'].str.contains('CANELEIRA', case=False) |
                df_inventario['Descr. Material'].str.contains('HALTER', case=False)))

    condicao7 = ((df_inventario['Grupo'] == 'ACESSORIOS') &
                (df_inventario['Categoria'].isin(['BASQUETE', 'FUTEBOL AMERICANO', 'FUTEVOLEI', 'HANDBALL', 
                                                    'VOLEI', 'AQUATICOS', 'COLETIVOS', 'FUTEBOL', 'LIFESTYLE', 
                                                    'NATACAO'])) &
                (df_inventario['Descr. Material'].str.contains('BOLA', case=False) |
                df_inventario['Descr. Material'].str.contains('MINI BOLA', case=False)))

    condicao8 = ((df_inventario['Grupo'] == 'EMBALAGENS') &
                (df_inventario['Categoria'].isin(['MARKETING', 'TODAS'])))

    condicao9 = ((df_inventario['Grupo'] == 'EQUIPAMENTOS') &
                (df_inventario['Categoria'] == 'TREINO') &
                (df_inventario['Descr. Material'].str.contains('KETTLEBELL', case=False)))

    condicao10 = ((df_inventario['Grupo'] == 'SERVICOS') &
                (df_inventario['Categoria'] == 'MARKETING') &
                (df_inventario['Descr. Material'].str.contains('KETTLEBELL', case=False)))

    condicao11 = ((df_inventario['Grupo'] == 'ALIMENTO'))

    condicao12 = ((df_inventario['DUPLICADOS'] == 'True'))
    
    


    return (condicao1, condicao2, condicao3, condicao4, condicao5, condicao6, 
            condicao7, condicao8, condicao9, condicao10, condicao11, condicao12)

    #---------------------------------------------------------------------------------------------------------------------------------------------------------------

    # Criando arquivo txt dos filtros dos itens que é contagem manual
    
def receber_manter_estoque_0(condicao1, condicao2, condicao3, condicao4, condicao5, condicao6, 
                            condicao7, condicao8, condicao9, condicao10, condicao11, condicao12):
    
    arquivo_txt = df_inventario[condicao1 | condicao2 | condicao3 | condicao4 | condicao5 | condicao6 | 
                                        condicao7 | condicao8 | condicao9 | condicao10 | condicao11 | condicao12]
    
    arquivo_txt = arquivo_txt.drop_duplicates(subset=['Material'])
    

    
    return arquivo_txt

# Consolidando Filtros
def consolidar(filtro_loja_nfp, filtro_loja_mb51, filtro_loja_rec, filtro_loja_mojix, df_inventario, arquivo_txt):
    # Definindo os nomes das colunas do df 'consolidado'
    colunas = ['Material', 'Movimento', 'Qtde']

    # Renomeando a coluna 'Diferença' para 'Qtde'
    df_inventario = df_inventario.rename(columns={'Qtd. Diferença': 'Qtde', 'CMV Empresa':'Custo Unitario'})


    # Criando o DataFrame vazio com as colunas especificadas
    consolidado = pd.DataFrame(columns=colunas)

    # Adicionando dados de NF PENDENTES no consolidado
    consolidado = pd.concat([consolidado, filtro_loja_nfp[['Material', 'Movimento', 'Qtde']]], ignore_index=True)

    # Adicionando dados da MB51 no consolidado
    consolidado = pd.concat([consolidado, filtro_loja_mb51[['Material', 'Movimento', 'Qtde']]], ignore_index=True)

    # Adicionando dados de DIVERGENCIA DE RECEBIMENTO no consolidado
    consolidado = pd.concat([consolidado, filtro_loja_rec[['Material', 'Movimento', 'Qtde']]], ignore_index=True)

    # Adicionando dados do MOJIX no consolidado
    consolidado = pd.concat([consolidado, filtro_loja_mojix[['Material', 'Movimento', 'Qtde']]], ignore_index=True)

    # Adicionando dados do ZTMM003 no consolidado
    consolidado = pd.concat([consolidado, df_inventario[['Material', 'Movimento', 'Qtde']]], ignore_index=True)
    
    materiais_excluir = arquivo_txt['Material'].unique()
    
    consolidado = consolidado[
        ~((consolidado['Material'].isin(materiais_excluir)) &
          (~consolidado['Movimento'].isin(['OMS', 'NF Pendente'])))]
    
    return consolidado, df_inventario

# Criando dinamica
def criar_dinamica(consolidado):
    # Cria tabela dinamica 
    dinamica = pd.pivot_table(consolidado, index="Material", columns="Movimento", values="Qtde", aggfunc="sum")

    # Exclui coluna vendas
    dinamica = dinamica.drop(columns=["Vendas"])

    # movendo colunas
    movendo_coluna = dinamica.pop('Diferenca Inventario')
    dinamica.insert(len(dinamica.columns), 'Diferenca Inventario', movendo_coluna)

    # filtrando o que for 0 e vazio na coluna 'Diferenca Inventario'
    filtro_dif_inv = dinamica.loc[(dinamica['Diferenca Inventario'] == 0) | 
                                    (dinamica['Diferenca Inventario'].isna())]

    # exclui o que foi filtrado em filtro dif inv
    dinamica = dinamica.drop(filtro_dif_inv.index)

    # Remove colunas totalmente vazias do DF
    dinamica = dinamica.dropna(axis=1, how="all")

    # Adiciona a coluna analise do DF Dinamica
    dinamica["Analise"] = None

    return dinamica

# Analises
def analises(dinamica):
    # ANALISES

    # Verifica em todas as colunas os valores que estão vazios e classifica como sem motivo
    # transforme colunas no formato numerico
    for coluna in dinamica.columns[:-1]:
        dinamica[coluna] = pd.to_numeric(dinamica[coluna], errors='coerce')

    sem_motivo = dinamica.copy()

    for coluna in dinamica.columns[:-2]:
        sem_motivo = sem_motivo[sem_motivo[coluna].isna()]

    dinamica.loc[dinamica.index.isin(sem_motivo.index), "Analise"] = "Sem motivo"

    del sem_motivo

    # Verifica se é NF Pendente
    if "NF Pendente" in dinamica.columns:
        ganho_nf = dinamica[dinamica['NF Pendente'].notna() & (dinamica['Diferenca Inventario'] > 0)]
        dinamica.loc[ganho_nf.index, "Analise"] = "NF Pendente"
        del ganho_nf

    # Verifica se é OMS
    if "OMS" in dinamica.columns:
        ganho_oms = dinamica[dinamica['OMS'].notna() & (dinamica['Diferenca Inventario'] > 0)]
        dinamica.loc[ganho_oms.index, "Analise"] = "OMS"
        del ganho_oms

    # Verifica se é Ganho de Divergencia de Recebimento
    if "Ganho Diverg. Receb." in dinamica.columns:
        ganho_dv_receb = dinamica[dinamica['Ganho Diverg. Receb.'].notna()]
        i_iguais = ganho_dv_receb['Ganho Diverg. Receb.'] == ganho_dv_receb['Diferenca Inventario']
        dinamica.loc[ganho_dv_receb[i_iguais].index, "Analise"] = "Ganho Diverg. Receb."
        del ganho_dv_receb
        del i_iguais

    # Verifica se é Perda de Divergencia de Recebimento
    if "Perda Diverg. Receb." in dinamica.columns:
        perda_dv_receb = dinamica[dinamica['Perda Diverg. Receb.'].notna()]
        i_iguais = perda_dv_receb['Perda Diverg. Receb.'] == -perda_dv_receb['Diferenca Inventario']
        dinamica.loc[perda_dv_receb[i_iguais].index, "Analise"] = "Perda Diverg. Receb."
        del perda_dv_receb
        del i_iguais

    # Verificar se é reversão de inventários
    if "Inventarios" in dinamica.columns:
        inventarios = dinamica[dinamica['Inventarios'].notna()]
        i_iguais = inventarios['Inventarios'] == -inventarios['Diferenca Inventario']
        dinamica.loc[inventarios[i_iguais].index, "Analise"] = "Reversão Inventarios"
        del inventarios
        del i_iguais

    # Verifica se é falta ou sobra de etiqueta
    if "Devolução Cliente" in dinamica.columns and "Mojix" in dinamica.columns:
        i_com_mojix = dinamica[dinamica['Devolução Cliente'].notna() & dinamica['Analise'].isna()]
        falta_etq = (i_com_mojix['Devolução Cliente'] - i_com_mojix['Mojix'].fillna(0)) == -i_com_mojix['Diferenca Inventario']
        dinamica.loc[falta_etq.index[falta_etq], "Analise"] = "Falta por Etiqueta"
        del i_com_mojix
        del falta_etq
    else:
        i_sem_mojix = dinamica['Devolução Cliente'] == -dinamica['Diferenca Inventario']
        dinamica.loc[i_sem_mojix & dinamica['Analise'].isna(), "Analise"] = "Falta por Etiqueta"
        del i_sem_mojix

    if "Mojix" in dinamica.columns:
        i_mojix = dinamica[dinamica['Mojix'].notna() & dinamica['Analise'].isna()]
        sobra_etq = i_mojix['Mojix'] == i_mojix['Diferenca Inventario']
        dinamica.loc[sobra_etq.index[sobra_etq], "Analise"] = "Sobra por Etiqueta"
        del i_mojix
        del sobra_etq

    # Verifica se é "possivelmente" por reversão de inventario
    if "Inventarios" in dinamica.columns:
        # Filtra as linhas onde 'Inventarios' não é nulo e 'Analise' é nulo
        i_possivel_p = dinamica[dinamica['Inventarios'].notna() & dinamica['Analise'].isna()]
        
        # Condições para atualizar 'Analise'
        sipa_perda_1 = (i_possivel_p['Inventarios'] > 0) & (i_possivel_p['Diferenca Inventario'] < 0) & (i_possivel_p['Diferenca Inventario'].abs() < i_possivel_p['Inventarios'].abs())
        sipa_perda_2 = (i_possivel_p['Inventarios'] < 0) & (i_possivel_p['Diferenca Inventario'] > 0) & (i_possivel_p['Diferenca Inventario'].abs() < i_possivel_p['Inventarios'].abs())
        
        # Atualiza a coluna 'Analise' no DataFrame original baseado nos índices filtrados
        dinamica.loc[i_possivel_p.index[sipa_perda_1 | sipa_perda_2], 'Analise'] = "Possivelmente Reversão Inventarios"
        del i_possivel_p
        del sipa_perda_1
        del sipa_perda_2

    # Verifica se é "possivelmente" por Ganho Divergencia de Recebimento
    if "Ganho Diverg. Receb." in dinamica.columns:
        
        gan_y_col = dinamica[dinamica['Analise'].isna()]
        
    # Verifica se há dados na coluna "Ganho Diverg. Receb."
        if gan_y_col['Ganho Diverg. Receb.'].isna().all():
            pass
        else:
            # Condições para atualizar 'Analise'
            cond_ganho_1 = (gan_y_col['Ganho Diverg. Receb.'] > 0) & (gan_y_col['Diferenca Inventario'] > 0) & (gan_y_col['Ganho Diverg. Receb.'] > gan_y_col['Diferenca Inventario'])
            
            # Atualiza a coluna 'Analise' onde a condição é satisfeita
            dinamica.loc[gan_y_col.index[cond_ganho_1], 'Analise'] = "Possivelmente Ganho Diverg. Receb."
            
            del cond_ganho_1
        del gan_y_col

    # Verifica se é "possivelmente" por Perda Divergencia de Recebimento
    if "Perda Diverg. Receb." in dinamica.columns:
        
        perd_y_col = dinamica[dinamica['Analise'].isna()]
        
    # Verifica se há dados na coluna "Perda Diverg. Receb."
        if perd_y_col['Perda Diverg. Receb.'].isna().all():
            pass
        else:
            # Condições para atualizar 'Analise
            cond_perda_1 = (perd_y_col['Perda Diverg. Receb.'] > 0) & (perd_y_col['Diferenca Inventario'] < 0) & (perd_y_col['Perda Diverg. Receb.'].abs() > perd_y_col['Diferenca Inventario'].abs())
            
            # Atualiza a coluna 'Analise' onde a condição é satisfeita
            dinamica.loc[perd_y_col.index[cond_perda_1], 'Analise'] = "Possivelmente Perda Diverg. Receb."
            
            del cond_perda_1
        del perd_y_col
        
    # Verifica se é "possivelmente" a Falta por Etiqueta
    if "Devolução Cliente" in dinamica.columns:
        
        dev_cli = dinamica[dinamica['Analise'].isna()]
        
        if dev_cli['Devolução Cliente'].isna().all():
            pass
        else:
            cond_dev = (dev_cli['Devolução Cliente'] > 0) & (dev_cli['Diferenca Inventario'] < 0) & (dev_cli['Devolução Cliente'].abs() > dev_cli['Diferenca Inventario'].abs())
            
            dinamica.loc[dev_cli.index[cond_dev], 'Analise'] = "Possivelmente Falta por Etiqueta"
            
            del cond_dev
        del dev_cli
        
    # Verifica se é "possivelmente" a Sobra por Etiqueta
    if "Mojix" in dinamica.columns:
        
        index_mojix = dinamica[dinamica['Analise'].isna()]
        
        if index_mojix['Mojix'].isna().all():
            pass
        else:
            cond_mojix = (index_mojix['Mojix'] > 0) & (index_mojix['Diferenca Inventario'] > 0) & (index_mojix['Mojix'] > index_mojix['Diferenca Inventario'])
            
            dinamica.loc[index_mojix.index[cond_mojix], 'Analise'] = "Possivelmente Sobra por Etiqueta"
            
            del cond_mojix
        del index_mojix

    # Verifica se existe motivos desconhecidos, analise e nomeia com o motivo. Depois filtra o resto e nomeia como analisar

    # Lista de colunas conhecidas
    colunas_conhecidas = ['Devolução Cliente', 'Ganho Diverg. Receb.', 'Inventarios', 'Mojix', 'NF Pendente', 'OMS', 'Perda Diverg. Receb.']

    # Pegar as colunas que não estão na lista de colunas conhecidas
    colunas_desconhecidas = [coluna for coluna in dinamica.columns[:-2] if coluna not in colunas_conhecidas]

    # Se houver colunas não conhecidas, filtrar as linhas vazias nessas colunas
    if colunas_desconhecidas:
        index_col_desconhecida = dinamica[dinamica['Analise'].isna()]
        for coluna in colunas_desconhecidas: 
            cond_desconhecida = (index_col_desconhecida[coluna].abs() == index_col_desconhecida['Diferenca Inventario'].abs())
            for idx in index_col_desconhecida[cond_desconhecida].index:
                if pd.isna(dinamica.at[idx, 'Analise']):
                    dinamica.at[idx, 'Analise'] = coluna
                else:
                    dinamica.at[idx, 'Analise'] += '/' + coluna  
                        
        index_col_desconhecida_1 = dinamica[dinamica['Analise'].isna()]
        dinamica.loc[index_col_desconhecida_1.index, 'Analise'] = "Analisar"
        
        del index_col_desconhecida
        del cond_desconhecida
    else:
        pass

    
    return dinamica

# Inserindo valores nas analises
def inserir_valores(dinamica, df_inventario):
    #  Incluindo valores na analise

    # Transformando a pivot_table em um DataFrame regular
    df_dinamica = dinamica.reset_index()

    # Verificar se a coluna 'Material' existe em ambos os DataFrames
    if 'Material' in df_dinamica.columns and 'Material' in df_inventario.columns:
        try:
            # Fazendo a junção com merge, garantindo que a chave de junção está correta
            df_dinamica = pd.merge(df_dinamica, df_inventario[['Material', 'Descr. Material', 'Qtd. Contabilistica', 'Custo Unitario']], on='Material', how='left')
            # tirar os espaços das celulas
            df_dinamica['Custo Unitario'] = df_dinamica['Custo Unitario'].str.strip()
            # trocar a virgula por ponto (no pandas o divisor decimal é panda 'tipo americano')
            df_dinamica['Custo Unitario'] = df_dinamica['Custo Unitario'].str.replace(',', '.')
            # transformar a coluna em float
            df_dinamica['Custo Unitario'] = df_dinamica['Custo Unitario'].astype('float64') 
            # criar a coluna TOTAl multplicando a diferença x custo
            df_dinamica['Valor Total'] = df_dinamica['Diferenca Inventario'] * df_dinamica['Custo Unitario']
            # tirando a coluna desejada
            mover_coluna = df_dinamica.pop('Descr. Material')
            # inserindo a coluna desejada no indice desejado
            df_dinamica.insert(1, 'Descr. Material', mover_coluna)
            

        except Exception as e:
            print("Erro ao juntar os DataFrames:", e)
    else:
        print("Coluna de junção 'Material' não encontrada em um ou ambos os DataFrames")
    return df_dinamica


def formatar_txt(arquivo_txt, df_dinamica, loja_analisada):

    # Lista com os índices das colunas que irá selecionar
    indices_colunas_txt = [4, 5, 18, 22]

    # Selecionando as colunas pelo índice
    arquivo_txt = arquivo_txt.iloc[:, indices_colunas_txt]

    # Renomeando colunas
    arquivo_txt = arquivo_txt.rename(columns={arquivo_txt.columns[2]: 'Custo Unitario'})

    manter_estoque = df_dinamica['Analise'].isin(['NF Pendente', 'OMS'])
    filtro_manter_estoque = df_dinamica[manter_estoque]

    colunas_desejadas = ['Material', 'Descr. Material', 'Custo Unitario', 'Qtd. Contabilistica']
    filtro_manter_estoque = filtro_manter_estoque[colunas_desejadas]

    arquivo_txt = pd.concat([arquivo_txt, filtro_manter_estoque])

    # Converte a coluna 'Qtd. Contabilistica' para numérico, substituindo erros por NaN
    arquivo_txt['Qtd. Contabilistica'] = pd.to_numeric(arquivo_txt['Qtd. Contabilistica'], errors='coerce')


    # Converte valores negativos de 'Qtd. Contabilistica' para 0
    arquivo_txt['Qtd. Contabilistica'] = arquivo_txt['Qtd. Contabilistica'].apply(lambda x: max(x, 0) if pd.notnull(x) else x)

    # Definir os valores fixos a serem concatenados
    zero = '0'

    # Criar a nova coluna concatenando os valores desejados
    arquivo_txt['Concatenado'] = zero + "|" + loja_analisada + "|" + arquivo_txt['Material'].astype(str) + "|" + arquivo_txt['Qtd. Contabilistica'].astype(int).astype(str)

    return arquivo_txt


# resumindo analises
def resumo_analise(df_dinamica):
    # Criando RESUMO analise

    # Substituir NaN por zero ou outro valor padrão
    df_dinamica = df_dinamica.fillna(0)

    # Criar a tabela dinâmica
    resultados = df_dinamica.pivot_table(index='Analise', values=['Diferenca Inventario', 'Valor Total'], aggfunc='sum', margins=True, margins_name='Total')

    # Transformar o índice da tabela dinâmica em coluna
    resultados = resultados.reset_index()
    
    return df_dinamica, resultados


# Salvando Resultado    
def salvar_resultados(df_dinamica, resultados, arquivo_txt, loja_analisada, itens_negativos):
    
    hora = datetime.now().strftime('%H_%M')

    # Salvando o DataFrame original e a tabela dinâmica em um arquivo Excel com múltiplas guias
    with pd.ExcelWriter(f'Resultado_Analise_{loja_analisada}_{hora}.xlsx', engine='xlsxwriter') as writer:
        df_dinamica.to_excel(writer, sheet_name='Analise Completa', index=False)
        resultados.to_excel(writer, sheet_name='Resumo Analise', index=False)
        arquivo_txt.to_excel(writer, sheet_name='TXT', index=False)
        itens_negativos.to_excel(writer, sheet_name='Negativos', index=False)

        # Acessando o workbook e as planilhas
        workbook = writer.book
        ws_data = writer.sheets['Analise Completa']
        ws_pivot = writer.sheets['Resumo Analise']
        ws_txt = writer.sheets['TXT']
        ws_negativos = writer.sheets['Negativos']
    
        # Adicionando formatação
        header_format = workbook.add_format({'bold': True, 'bg_color': '#4F81BD', 'font_color': 'white', 'border': 1, 'align': 'center', 'valign': 'vcenter'})
        cell_format = workbook.add_format({'border': 1, 'align': 'center', 'valign': 'vcenter'})
        currency_format = workbook.add_format({'border': 1, 'align': 'center', 'valign': 'vcenter', 'num_format': 'R$ #,##0.00'})  # Formato moeda Reais

        # Aplicando formatação ao cabeçalho da aba 'Analise Completa'
        for col_num, value in enumerate(df_dinamica.columns.values):
            ws_data.write(0, col_num, value, header_format)

        # Aplicando formatação às células da aba 'Analise Completa'
        for row_num in range(1, len(df_dinamica) + 1):
            for col_num in range(len(df_dinamica.columns)):
                if df_dinamica.columns[col_num] in ['Valor Total', 'Custo Unitario']:
                    ws_data.write(row_num, col_num, df_dinamica.iloc[row_num - 1, col_num], currency_format)
                else:
                    ws_data.write(row_num, col_num, df_dinamica.iloc[row_num - 1, col_num], cell_format)

        # Aplicando formatação ao cabeçalho da aba 'Resumo Analise'
        for col_num, value in enumerate(resultados.columns):
            ws_pivot.write(0, col_num, value, header_format)
        
        # Aplicando formatação às células da aba 'Resumo Analise'
        for row_num in range(1, len(resultados) + 1):
            for col_num in range(len(resultados.columns)):
                if resultados.columns[col_num] == 'Valor Total':
                    ws_pivot.write(row_num, col_num, resultados.iloc[row_num - 1, col_num], currency_format)
                else:
                    ws_pivot.write(row_num, col_num, resultados.iloc[row_num - 1, col_num], cell_format)

        # Formatando a linha que contém 'Total' na coluna 'A'
        for row_num in range(1, len(resultados) + 1):
            if resultados.iloc[row_num - 1, 0] == 'Total':  # Verifica se a coluna 'A' contém 'Total'
                for col_num in range(len(resultados.columns)):
                    ws_pivot.write(row_num, col_num, resultados.iloc[row_num - 1, col_num], header_format)

        # Aplicando formatação ao cabeçalho da aba 'TXT'
        for col_num, value in enumerate(arquivo_txt.columns):
            ws_txt.write(0, col_num, value, header_format)

        # Aplicando formatação ao cabeçalho da aba 'Negativos'
        for col_num, value in enumerate(itens_negativos.columns):
            ws_negativos.write(0, col_num, value, header_format)

        # Desativar a exibição das linhas de grade
        ws_data.hide_gridlines(option=2)
        ws_pivot.hide_gridlines(option=2)
        ws_txt.hide_gridlines(option=2)
        ws_negativos.hide_gridlines(option=2)
        
        print('\nAnalise Finalizada :D')


        
# Importar diretorio do arquivo mojix
def importar_mojix(arquivo_mojix):
    try:
        # Tentativa com separador ";"
        df_mojix = pd.read_csv(arquivo_mojix, encoding='latin1', sep=';', converters={'SKU': str})
        
        # Verifica se a importação foi bem-sucedida
        if df_mojix.empty or len(df_mojix.columns) <= 1:
            raise ValueError("Leitura incorreta do arquivo com separador ';'. Tentando com ','...")

        print("\nLeitura Ok.")
        return df_mojix

    except (pd.errors.ParserError, ValueError) as e:
        print(str(e))
        try:
            # Tentativa com separador ","
            df_mojix = pd.read_csv(arquivo_mojix, encoding='latin1', sep=',', converters={'SKU': str})

            # Verifica se a importação foi bem-sucedida
            if df_mojix.empty or len(df_mojix.columns) <= 1:
                raise ValueError("Leitura incorreta do arquivo com separador ','.")

            print("\nLeitura Ok.")
            return df_mojix
        
        except Exception as e:
            print(f"Erro ao tentar ler o arquivo: {e}")
            return None


# Importar diretorio do arquivo contagem manual
def importar_contag_manual(arquivo_manual):
    
    arquivo_contag_manual = pd.read_excel(arquivo_manual)

    # Renomeando colunas
    arquivo_contag_manual = arquivo_contag_manual.rename(columns={arquivo_contag_manual.columns[0]: 'Material'})
    arquivo_contag_manual = arquivo_contag_manual.rename(columns={arquivo_contag_manual.columns[1]: 'Qtd. Contabilistica'})
    arquivo_contag_manual = arquivo_contag_manual[arquivo_contag_manual['Qtd. Contabilistica'] != 0]

    return arquivo_contag_manual

# MAIN

# Lojas
lojas = []

# TEMA
sg.theme('BrownBlue')

# FONTE
fonte_padrao = 'Calibri'

# LAYOUT DO SISTEMA
layout1 = [
    [sg.Text("INVENTÁRIO LOJAS", font=(fonte_padrao, 40, 'bold'), justification='center', expand_x=True)],
    [sg.Text("\nInsira as lojas com 4 digitos. EX.: (0000)", font=(fonte_padrao, 14, 'bold'))],
    [sg.Text('Loja:', font=(fonte_padrao, 11, 'bold')), sg.InputText(key='input', enable_events=True, focus=True, size=(10, 1)),
     sg.Button('Adicionar Loja', font=(fonte_padrao, 11, 'bold')), sg.Button('Remover Loja', font=(fonte_padrao, 11, 'bold'))],
    [sg.Text("\nSelecione a data mais antiga do último inventário entre as lojas.", font=(fonte_padrao, 14, 'bold'))],
    [sg.Text('Data:', font=(fonte_padrao, 11, 'bold')), sg.InputText(key='data_input', size=(10, 1)),
     sg.CalendarButton('Calendário', target='data_input', font=(fonte_padrao, 11, 'bold'), format='%d/%m/%Y')],
    [sg.Text("\nResumo:", font=(fonte_padrao, 14, 'bold'))],
    [sg.Output(size=(80, 7), key='-OUTPUT-', pad=(0, 10))],
    [sg.Button("Extrair Bases", font=(fonte_padrao, 11, 'bold')), sg.Button('Analisar Lojas', font=(fonte_padrao, 11, 'bold')), sg.Button("Sair", font=(fonte_padrao, 11, 'bold'), size=(10,1), button_color="red")]
]

# INFORMAÇÕES DA JANELA
janela1 = sg.Window("INVENTÁRIO LOJAS", layout1, size=(800, 600), resizable=True, finalize=True, icon="img_gruposbf.ico", font=fonte_padrao)

# PEGANDO DATA ANTERIOR A HOJE
data_hoje = date.today()
data_hoje = data_hoje.strftime("%d%m%Y")

data_final = date.today() - timedelta(1)
data_final = data_final.strftime("%d%m%Y")


#--------------------------------------------------------------------------------------------------------------
# INICIANDO SAP GUI AUTOMAÇÃO

janela_prod = verificar_sap()

if janela_prod:
    
    abrir_nova_sessao()
else:
    
    acessar_sap()

#--------------------------------------------------------------------------------------------------------------
# EXECUTANDO SISTEMA
while True:
    event, values = janela1.read()

    if event == sg.WIN_CLOSED or event == "Sair":
        confirm_exit = sg.popup_yes_no("Você deseja realmente sair?", title="INVENTÁRIO LOJAS", font=fonte_padrao, icon=sg.SYSTEM_TRAY_MESSAGE_ICON_WARNING)
        if confirm_exit == "Yes":
            break
# -------------------------------------------------------------------------------------------------------------------------------------------------------
    # Botão e 2° janela para inserir macro de analise

    if event == 'Analisar Lojas':
        # Defina o layout da segunda janela
        layout2 = [
            [sg.Text("INVENTÁRIO LOJAS", font=(fonte_padrao, 40, 'bold'), justification='center', expand_x=True)],
            [sg.Text("")],
            [sg.Combo(lojas, default_value='Selecionar Loja', size=(20, 1), key='lista_suspensa'),
            sg.Button("Importar Mojix", font=(fonte_padrao, 11, 'bold'), key="Importar Mojix"), sg.Checkbox("Sem Mojix", font=(fonte_padrao, 11, 'bold'), key='Mojix Vazio')],
            [sg.Text("\nResumo:", font=(fonte_padrao, 11, 'bold'))],
            [sg.Output(size=(60, 7), key='-OUTPUT-', pad=(0, 10))],
            [sg.Button('Iniciar Analise', font=(fonte_padrao, 11, 'bold'), key='Analise'), sg.Button('Fechar', font=(fonte_padrao, 11, 'bold'), size=(10, 1), button_color="red")]
        ]

        janela2 = sg.Window("INVENTÁRIO LOJAS", layout2, size=(700, 500), resizable=True, finalize=True, icon="img_gruposbf.ico", font=fonte_padrao)

        # Criando o df mojix em branco como padrão.
        columns = ['Encoded Date', 'Department Name', 'Brand', 'SKU', 'Product Name', 
        'Zone', 'Store ID', 'Item Status', 'EPC', 'OriginalEPC (barcode)', 'Image URL']
        
        df_mojix = pd.DataFrame(columns=columns)  # Define o df_mojix vazio
        
        # Loop de eventos para a segunda janela
        while True:
            event2, values2 = janela2.read()
            
        # evento para quando for fechar a janela
            if event2 == sg.WINDOW_CLOSED or event2 == 'Fechar':
                break
            
        # evento de importar a base do mojix
            if event2 == "Importar Mojix":
                arquivo_mojix = sg.popup_get_file("Selecione o arquivo desejado: ", icon="img_gruposbf.ico")
                if arquivo_mojix:
                    print(f"Arquivo selecionado: {arquivo_mojix}")
                    
                    df_mojix = importar_mojix(arquivo_mojix)
                    if df_mojix is not None:
                        print("\nImportação Mojix concluída com sucesso.")
                    else:
                         sg.popup_error("Erro ao importar o arquivo Mojix. Verifique o formato e tente novamente.", icon=sg.SYSTEM_TRAY_MESSAGE_ICON_WARNING)
                       
                            
        # evento de quando clicar em analise
            if event2 == "Analise":
                # Armazena a loja selecionada na variável loja_analisada
                loja_analisada = values2['lista_suspensa']
                
                if not loja_analisada:
                    sg.popup_error("\nNenhuma loja selecionada, Por favor, selecione uma loja para análise.", icon=sg.SYSTEM_TRAY_MESSAGE_ICON_WARNING)
                    break
                    
                elif loja_analisada == 'Selecionar Loja':
                    sg.popup_error("\nNenhuma loja selecionada, Por favor, selecione uma loja para análise.", icon=sg.SYSTEM_TRAY_MESSAGE_ICON_WARNING)
                    break
                
                else:
                    print(f"\nLoja selecionada para análise: {loja_analisada}")
                    
                if df_mojix is None:
                    sg.popup_error("\nNenhuma base Mojix foi definida.")

                
                # Verificando a data do último inventário com uma janela pop-up
                layout_data = [
                    [sg.Text('Por favor, insira a data do último inventário (dd/mm/aaaa):')],
                    [sg.Input(key='data_inventario')],
                    [sg.Button('OK')]
                ]
                window_data = sg.Window('Data do Último Inventário', layout_data)

                while True:
                    event_data, values_data = window_data.read()
                    if event_data == sg.WIN_CLOSED:
                        sg.popup('A operação foi cancelada.', icon="img_gruposbf.ico")
                        window_data.close()
                        break 
                    elif event_data == 'OK':
                        data_analisada = values_data['data_inventario']
                        try:
                            datetime.strptime(data_analisada, '%d/%m/%Y')
                            print('Data válida:', data_analisada)
                            break
                        except ValueError:
                            sg.popup('Data inválida. Tente novamente.', icon=sg.SYSTEM_TRAY_MESSAGE_ICON_WARNING)

                window_data.close()

                # Verificando se tem contagem manual com uma janela pop-up
                layout_coleta = [
                    [sg.Text('O inventário tem coleta manual?')],
                    [sg.Radio('SIM', 'coleta', key='coleta_manual_sim'), sg.Radio('NÃO', 'coleta', key='coleta_manual_nao')],
                    [sg.Button('OK')]
                ]
                window_coleta = sg.Window('Coleta Manual', layout_coleta, icon="img_gruposbf.ico")
                
                while True:
                    event_coleta, values_coleta = window_coleta.read()
                    if event_coleta == sg.WIN_CLOSED:
                        sg.popup('A operação foi cancelada.')
                        window_coleta.close()
                        break
                    elif event_coleta == 'OK':
                        coleta_manual = 'SIM' if values_coleta.get('coleta_manual_sim') else 'NÃO'
                        print('Coleta manual:', coleta_manual)
                        window_coleta.close()  # Fechando a janela antes de iniciar a execução das funções
                        
                        if coleta_manual == 'SIM':
                            arquivo_manual = sg.popup_get_file("Selecione um arquivo:", icon="img_gruposbf.ico")
                            if arquivo_manual:                        
                                print(f"Arquivo selecionado: {arquivo_manual}")
                                
                                arquivo_contag_manual = importar_contag_manual(arquivo_manual)
                                
                                df_inventario, itens_negativos = ext_ztmm003(loja_analisada)
                                
                                filtro_loja_nfp, filtro_loja_mb51, filtro_loja_rec, filtro_loja_mojix, df_inventario = tratar_bases(df_inventario, df_nfp, df_mojix, df_mb51, df_rec)
                                
                                condicao13 = filtros_manter_estoque_1(df_inventario)
                                
                                arquivo_txt, itens_negativos = receber_manter_estoque_1(condicao13, arquivo_contag_manual)
                                
                                consolidado, df_inventario = consolidar(filtro_loja_nfp, filtro_loja_mb51, filtro_loja_rec, filtro_loja_mojix, df_inventario, arquivo_txt)
                                
                                dinamica = criar_dinamica(consolidado)
                                dinamica = analises(dinamica)
                                df_dinamica = inserir_valores(dinamica, df_inventario)
                                
                                arquivo_txt = formatar_txt(arquivo_txt, df_dinamica, loja_analisada)
                                df_dinamica, resultados = resumo_analise(df_dinamica)
                                salvar_resultados(df_dinamica, resultados, arquivo_txt, loja_analisada, itens_negativos)                                
                                
                        elif coleta_manual == 'NÃO':
                            try:
                                df_inventario, itens_negativos = ext_ztmm003(loja_analisada)
                            except:
                                break
                            
                            filtro_loja_nfp, filtro_loja_mb51, filtro_loja_rec, filtro_loja_mojix, df_inventario = tratar_bases(df_inventario, df_nfp, df_mojix, df_mb51, df_rec)
                            
                            condicao1, condicao2, condicao3, condicao4, condicao5, condicao6, condicao7, condicao8, condicao9, condicao10, condicao11, condicao12 = filtros_manter_estoque_0(df_inventario)
                            
                            arquivo_txt = receber_manter_estoque_0(condicao1, condicao2, condicao3, condicao4, condicao5, condicao6, 
                            condicao7, condicao8, condicao9, condicao10, condicao11, condicao12)
                            
                            consolidado, df_inventario = consolidar(filtro_loja_nfp, filtro_loja_mb51, filtro_loja_rec, filtro_loja_mojix, df_inventario, arquivo_txt)
                            
                            dinamica = criar_dinamica(consolidado)
                            dinamica = analises(dinamica)
                            df_dinamica = inserir_valores(dinamica, df_inventario)
                            
                            arquivo_txt = formatar_txt(arquivo_txt, df_dinamica, loja_analisada)
                            df_dinamica, resultados = resumo_analise(df_dinamica)
                            salvar_resultados(df_dinamica, resultados, arquivo_txt, loja_analisada, itens_negativos)

                        break
                

        janela2.close()
# -----------------------------------------------------------------------------------------------------------------------------------------------------------

    if event == "Adicionar Loja":
        loja = get_input(values)
        if loja:
            lojas.append(loja)
            janela1['-OUTPUT-'].print(f"Loja Adicionada: {loja}")
        else:
            sg.popup_error("Por favor, insira um número de loja válido. (0004) dígitos.", title='Erro', icon="img_gruposbf.ico")
        janela1['input'].update('')

    if event == "Extrair Bases":
        
        data_inicial = values["data_input"]
        data_inicial = data_inicial.replace("/", "")
        
        if not data_inicial or not lojas:
            sg.popup_error('Campo data e/ou loja vazio(s). Por favor, preencher os campos.', title='Erro!!', icon="img_gruposbf.ico")
        
        else:    
            print("\nExtração das bases iniciada...")
            
            df_mb51 = ext_mb51()
            
            df_nfp = ext_nfp()
            
            df_rec = ext_dreceb()
            
            gerar_macros()

    if event == "Remover Loja":
        if lojas:
            removed_item = lojas.pop()
            janela1['-OUTPUT-'].print(f"Loja removida: {removed_item}")
            janela1['-OUTPUT-'].print("Lojas restantes:", ', '.join(lojas))

# Armazenando o stdout original
original_stdout = sys.stdout
sys.stdout = original_stdout

janela1.close()



KeyError: 'Valor Total'

In [9]:
df_dinamica_teste = df_dinamica

In [12]:
df_dinamica_teste = pd.merge(df_dinamica_teste, df_inventario, on='Material', how='left')


In [13]:
df_dinamica_teste

,Material,672,Devolução Cliente,Ganho Diverg. Receb.,Inventarios,NF Pendente,OMS,Perda Diverg. Receb.,Recebimento CD/LOJA,Saída de Transferência,...,Mont.Custo Venda,Estoque atual,1. Contagem,Qtd. 1 Recontagem,Qtd. 2 Recontagem,Qtd. 3 Recontagem,Qtd. 4 Recontagem,STATUS,Movimento,DUPLICADOS
0,512114001519,NaN,NaN,NaN,0.0,NaN,NaN,1.0,1.0,NaN,...,299.99,3.0,0.0,0.0,0.0,0.0,0.0,"Contado, corrigido",Diferenca Inventario,False
1,512117003138,NaN,NaN,NaN,1.0,NaN,NaN,NaN,3.0,NaN,...,159.99,3.0,0.0,0.0,0.0,0.0,0.0,"Contado, corrigido",Diferenca Inventario,False
2,519439022021,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,...,-109.99,3.0,0.0,0.0,0.0,0.0,0.0,"Contado, corrigido",Diferenca Inventario,False
3,519439022045,NaN,1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,...,-109.99,8.0,0.0,0.0,0.0,0.0,0.0,"Contado, corrigido",Diferenca Inventario,False
4,533497002733,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,199.99,1.0,1.0,0.0,0.0,0.0,0.0,"Contado, corrigido",Diferenca Inventario,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,990965012043,NaN,NaN,NaN,-2.0,NaN,NaN,2.0,6.0,NaN,...,-399.98,4.0,0.0,0.0,0.0,0.0,0.0,"Contado, corrigido",Diferenca Inventario,False
1524,990987192044,NaN,NaN,1.0,1.0,NaN,NaN,NaN,5.0,NaN,...,349.99,6.0,0.0,0.0,0.0,0.0,0.0,"Contado, corrigido",Diferenca Inventario,False
1525,991091020407,NaN,NaN,NaN,0.0,NaN,-1.0,NaN,NaN,NaN,...,699.99,1.0,1.0,0.0,0.0,0.0,0.0,"Contado, corrigido",Diferenca Inventario,False
1526,991246022034,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,...,139.99,3.0,0.0,0.0,0.0,0.0,0.0,"Contado, corrigido",Diferenca Inventario,False


: 